<a href="https://colab.research.google.com/github/kazimoto11/Uber_Sendy_ML_Ug_thesis/blob/master/Finale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import tensorflow as tf

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
import io

Order placement, confirmation and pickup is done at the same day - (Almost all the times)

In [0]:
train = pd.read_csv(io.BytesIO(uploaded['Train.csv']))
train.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10:27:30 AM,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745
1,Order_No_25375,User_Id_2285,Bike,3,Personal,12,5,11:16:16 AM,12,5,11:23:21 AM,12,5,11:40:22 AM,12,5,11:44:09 AM,12,5,12:17:22 PM,16,26.4,NaN,-1.351453,36.899315,-1.295004,36.814358,Rider_Id_856,1993
2,Order_No_1899,User_Id_265,Bike,3,Business,30,2,12:39:25 PM,30,2,12:42:44 PM,30,2,12:49:34 PM,30,2,12:53:03 PM,30,2,1:00:38 PM,3,NaN,NaN,-1.308284,36.843419,-1.300921,36.828195,Rider_Id_155,455
3,Order_No_9336,User_Id_1402,Bike,3,Business,15,5,9:25:34 AM,15,5,9:26:05 AM,15,5,9:37:56 AM,15,5,9:43:06 AM,15,5,10:05:27 AM,9,19.2,NaN,-1.281301,36.832396,-1.257147,36.795063,Rider_Id_855,1341
4,Order_No_27883,User_Id_1737,Bike,1,Personal,13,1,9:55:18 AM,13,1,9:56:18 AM,13,1,10:03:53 AM,13,1,10:05:23 AM,13,1,10:25:37 AM,9,15.4,NaN,-1.266597,36.792118,-1.295041,36.809817,Rider_Id_770,1214


In [0]:
train.describe()

,Platform Type,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Pickup - Weekday (Mo = 1),Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Time from Pickup to Arrival
count,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000,16835.000000,552.000000,21201.000000,21201.000000,21201.000000,21201.000000,21201.000000
mean,2.752182,15.653696,3.240083,15.653837,3.240225,15.653837,3.240225,15.653837,3.240225,15.653837,3.240225,9.506533,23.258889,7.905797,-1.281470,36.811264,-1.282581,36.811220,1556.920947
std,0.625178,8.798916,1.567295,8.798886,1.567228,8.798886,1.567228,8.798886,1.567228,8.798886,1.567228,5.668963,3.615768,17.089971,0.030507,0.037473,0.034824,0.044721,987.270788
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,11.200000,0.100000,-1.438302,36.653621,-1.430298,36.606594,1.000000
25%,3.000000,8.000000,2.000000,8.000000,2.000000,8.000000,2.000000,8.000000,2.000000,8.000000,2.000000,5.000000,20.600000,1.075000,-1.300921,36.784605,-1.301201,36.785661,882.000000
50%,3.000000,15.000000,3.000000,15.000000,3.000000,15.000000,3.000000,15.000000,3.000000,15.000000,3.000000,8.000000,23.500000,2.900000,-1.279395,36.807040,-1.284382,36.808002,1369.000000
75%,3.000000,23.000000,5.000000,23.000000,5.000000,23.000000,5.000000,23.000000,5.000000,23.000000,5.000000,13.000000,26.000000,4.900000,-1.257147,36.829741,-1.261177,36.829477,2040.000000
max,4.000000,31.000000,7.000000,31.000000,7.000000,31.000000,7.000000,31.000000,7.000000,31.000000,7.000000,49.000000,32.100000,99.100000,-1.147170,36.991046,-1.030225,37.016779,7883.000000


In [0]:
uploaded = files.upload()

Saving Test.csv to Test.csv


In [0]:
test = pd.read_csv(io.BytesIO(uploaded['Test.csv']))
test.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id
0,Order_No_19248,User_Id_3355,Bike,3,Business,27,3,4:44:10 PM,27,3,4:44:29 PM,27,3,4:53:04 PM,27,3,5:06:47 PM,8,NaN,NaN,-1.333275,36.870815,-1.305249,36.822390,Rider_Id_192
1,Order_No_12736,User_Id_3647,Bike,3,Business,17,5,12:57:35 PM,17,5,12:59:17 PM,17,5,1:20:27 PM,17,5,1:25:37 PM,5,NaN,NaN,-1.272639,36.794723,-1.277007,36.823907,Rider_Id_868
2,Order_No_768,User_Id_2154,Bike,3,Business,27,4,11:08:14 AM,27,4,11:25:05 AM,27,4,11:33:20 AM,27,4,11:57:54 AM,5,22.8,NaN,-1.290894,36.822971,-1.276574,36.851365,Rider_Id_26
3,Order_No_15332,User_Id_2910,Bike,3,Business,17,1,1:51:35 PM,17,1,1:53:27 PM,17,1,2:02:41 PM,17,1,2:16:52 PM,5,24.5,NaN,-1.290503,36.809646,-1.303382,36.790658,Rider_Id_685
4,Order_No_21373,User_Id_1205,Bike,3,Business,11,2,11:30:28 AM,11,2,11:34:45 AM,11,2,11:47:19 AM,11,2,11:56:04 AM,6,24.4,NaN,-1.281081,36.814423,-1.266467,36.792161,Rider_Id_858


In [0]:
test.describe()

,Platform Type,Placement - Day of Month,Placement - Weekday (Mo = 1),Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Pickup - Day of Month,Pickup - Weekday (Mo = 1),Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long
count,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,7068.000000,5631.000000,199.000000,7068.000000,7068.000000,7068.000000,7068.000000
mean,2.752122,15.621817,3.249859,15.621817,3.249859,15.621817,3.249859,15.621817,3.249859,9.458970,23.246120,6.651759,-1.282043,36.811148,-1.282255,36.811326
std,0.624000,8.776650,1.580926,8.776650,1.580926,8.776650,1.580926,8.776650,1.580926,5.575689,3.591161,14.085439,0.030044,0.037962,0.034236,0.043507
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,13.200000,0.100000,-1.433512,36.653828,-1.422652,36.639195
25%,3.000000,8.000000,2.000000,8.000000,2.000000,8.000000,2.000000,8.000000,2.000000,5.000000,20.600000,1.100000,-1.301201,36.784605,-1.300921,36.786546
50%,3.000000,15.000000,3.000000,15.000000,3.000000,15.000000,3.000000,15.000000,3.000000,8.000000,23.400000,2.900000,-1.281301,36.807219,-1.283219,36.807927
75%,3.000000,23.000000,5.000000,23.000000,5.000000,23.000000,5.000000,23.000000,5.000000,13.000000,25.900000,4.950000,-1.257274,36.829716,-1.261256,36.828195
max,4.000000,31.000000,7.000000,31.000000,7.000000,31.000000,7.000000,31.000000,7.000000,47.000000,32.100000,99.100000,-1.148352,36.991046,-1.030225,37.018958


In [0]:
#To use the time feature in the dataframe it is converted to an integer in hours

train['Pickup - Time'] = pd.to_datetime(train['Pickup - Time'])
train['Pickup - Time'] = train['Pickup - Time'].dt.hour
train['Pickup - Time'].head()

0    10
1    11
2    12
3     9
4    10
Name: Pickup - Time, dtype: int64

In [0]:
#To use the time feature in the dataframe it is converted to an integer in hours

test['Pickup - Time'] = pd.to_datetime(test['Pickup - Time'])
test['Pickup - Time'] = test['Pickup - Time'].dt.hour
test['Pickup - Time'].head()

0    17
1    13
2    11
3    14
4    11
Name: Pickup - Time, dtype: int64

In [0]:
uploaded = files.upload()

Saving Riders.csv to Riders.csv


In [0]:
riders = pd.read_csv(io.BytesIO(uploaded['Riders.csv']))

In [0]:
riders.head()

,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings
0,Rider_Id_396,2946,2298,14.0,1159
1,Rider_Id_479,360,951,13.5,176
2,Rider_Id_648,1746,821,14.3,466
3,Rider_Id_753,314,980,12.5,75
4,Rider_Id_335,536,1113,13.7,156


In [0]:
uploaded = files.upload()

Saving Uber_Sendy_hod.xlsx to Uber_Sendy_hod.xlsx


In [0]:
uber_sendy = pd.read_excel(io.BytesIO(uploaded['Uber_Sendy_hod.xlsx']))

In [0]:
uber_sendy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627697 entries, 0 to 627696
Data columns (total 9 columns):
Unnamed: 0                                  627697 non-null int64
Order No                                    627697 non-null object
sourceid                                    627697 non-null int64
dstid                                       627697 non-null int64
hod                                         627321 non-null float64
mean_travel_time                            627321 non-null float64
standard_deviation_travel_time              627321 non-null float64
geometric_mean_travel_time                  627321 non-null float64
geometric_standard_deviation_travel_time    627321 non-null float64
dtypes: float64(5), int64(3), object(1)
memory usage: 43.1+ MB


In [0]:
def ratings_feature(no_of_orders, age, avg_rating, no_of_ratings):
  ''' Creating a new feature by combining all columns fro rider_id datast'''

  rating = (age * avg_rating * no_of_ratings) / (no_of_orders * 1000)

  rating = pd.DataFrame(rating, columns = ['Rating'])
  
  return rating

In [0]:
Rating = ratings_feature(no_of_orders = riders['No_Of_Orders']
                         , age = riders['Age']
                         , avg_rating = riders['Average_Rating']
                         , no_of_ratings = riders['No_of_Ratings']
                         )

Rating.head()

,Rating
0,12.656941
1,6.276600
2,3.133436
3,2.925955
4,4.437880


In [0]:
ratings_feature = pd.concat([riders, Rating], axis = 1)
ratings_feature.head()

,Rider Id,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating
0,Rider_Id_396,2946,2298,14.0,1159,12.656941
1,Rider_Id_479,360,951,13.5,176,6.276600
2,Rider_Id_648,1746,821,14.3,466,3.133436
3,Rider_Id_753,314,980,12.5,75,2.925955
4,Rider_Id_335,536,1113,13.7,156,4.437880


In [0]:
features_train_new = pd.merge(train,ratings_feature, on = 'Rider Id')
features_train_new.head()

,Order No,User Id,Vehicle Type,Platform Type,Personal or Business,Placement - Day of Month,Placement - Weekday (Mo = 1),Placement - Time,Confirmation - Day of Month,Confirmation - Weekday (Mo = 1),Confirmation - Time,Arrival at Pickup - Day of Month,Arrival at Pickup - Weekday (Mo = 1),Arrival at Pickup - Time,Pickup - Day of Month,Pickup - Weekday (Mo = 1),Pickup - Time,Arrival at Destination - Day of Month,Arrival at Destination - Weekday (Mo = 1),Arrival at Destination - Time,Distance (KM),Temperature,Precipitation in millimeters,Pickup Lat,Pickup Long,Destination Lat,Destination Long,Rider Id,Time from Pickup to Arrival,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating
0,Order_No_4211,User_Id_633,Bike,3,Business,9,5,9:35:46 AM,9,5,9:40:10 AM,9,5,10:04:47 AM,9,5,10,9,5,10:39:55 AM,4,20.4,NaN,-1.317755,36.830370,-1.300406,36.829741,Rider_Id_432,745,1637,1309,13.8,549,6.058183
1,Order_No_27440,User_Id_2642,Bike,3,Personal,18,5,3:41:17 PM,18,5,3:41:30 PM,18,5,4:07:16 PM,18,5,16,18,5,5:01:43 PM,20,24.5,NaN,-1.326774,36.787807,-1.356237,36.904295,Rider_Id_432,2886,1637,1309,13.8,549,6.058183
2,Order_No_14170,User_Id_3339,Bike,3,Business,31,5,12:51:41 PM,31,5,1:12:49 PM,31,5,1:20:02 PM,31,5,13,31,5,2:16:43 PM,6,24.7,NaN,-1.255189,36.782203,-1.273412,36.818206,Rider_Id_432,2615,1637,1309,13.8,549,6.058183
3,Order_No_21727,User_Id_3523,Bike,3,Personal,2,2,7:12:10 AM,2,2,7:12:29 AM,2,2,7:35:24 AM,2,2,7,2,2,8:28:32 AM,18,15.2,NaN,-1.290315,36.757377,-1.223520,36.802061,Rider_Id_432,2986,1637,1309,13.8,549,6.058183
4,Order_No_8301,User_Id_87,Bike,2,Personal,22,2,10:40:58 AM,22,2,10:42:24 AM,22,2,10:56:00 AM,22,2,10,22,2,11:26:22 AM,7,19.2,NaN,-1.273524,36.799220,-1.300431,36.752427,Rider_Id_432,1602,1637,1309,13.8,549,6.058183


In [0]:
features_test_new = pd.merge(test,ratings_feature, on = 'Rider Id')
features_test_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7068 entries, 0 to 7067
Data columns (total 30 columns):
Order No                                7068 non-null object
User Id                                 7068 non-null object
Vehicle Type                            7068 non-null object
Platform Type                           7068 non-null int64
Personal or Business                    7068 non-null object
Placement - Day of Month                7068 non-null int64
Placement - Weekday (Mo = 1)            7068 non-null int64
Placement - Time                        7068 non-null object
Confirmation - Day of Month             7068 non-null int64
Confirmation - Weekday (Mo = 1)         7068 non-null int64
Confirmation - Time                     7068 non-null object
Arrival at Pickup - Day of Month        7068 non-null int64
Arrival at Pickup - Weekday (Mo = 1)    7068 non-null int64
Arrival at Pickup - Time                7068 non-null object
Pickup - Day of Month                   7068 n

In [0]:
features_new_merge = pd.concat([features_train_new,features_test_new])
features_new_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28269 entries, 0 to 7067
Data columns (total 34 columns):
Age                                          28269 non-null int64
Arrival at Destination - Day of Month        21201 non-null float64
Arrival at Destination - Time                21201 non-null object
Arrival at Destination - Weekday (Mo = 1)    21201 non-null float64
Arrival at Pickup - Day of Month             28269 non-null int64
Arrival at Pickup - Time                     28269 non-null object
Arrival at Pickup - Weekday (Mo = 1)         28269 non-null int64
Average_Rating                               28269 non-null float64
Confirmation - Day of Month                  28269 non-null int64
Confirmation - Time                          28269 non-null object
Confirmation - Weekday (Mo = 1)              28269 non-null int64
Destination Lat                              28269 non-null float64
Destination Long                             28269 non-null float64
Distance (KM)        

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [0]:
# Pre-selecting features for the linear regression model

#From train set a selected_training_features is created

# Merge testing only on training file

selected_training_features = features_train_new [['Order No','Distance (KM)',
'Pickup - Time','No_Of_Orders',	'Age', 'Average_Rating', 'No_of_Ratings',
'Rating', 'Time from Pickup to Arrival']]

#From test set a selected_test_features is created 
selected_test_features = features_test_new[['Order No', 'Distance (KM)', 
'Pickup - Time','No_Of_Orders',	'Age', 'Average_Rating', 'No_of_Ratings',
'Rating']]

#Concanetating the two selected features from the test and train data set
selected_features = pd.concat([selected_training_features,
selected_test_features], axis = 0, sort = False)

target_train_variables = features_new_merge['Time from Pickup to Arrival'].head(21201)

selected_features.describe()

,Distance (KM),Pickup - Time,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating,Time from Pickup to Arrival
count,28269.000000,28269.000000,28269.000000,28269.000000,28269.000000,28269.000000,28269.000000,21201.000000
mean,9.494641,12.864233,1698.576497,988.157522,13.882684,341.504333,2.784527,1556.920947
std,5.645726,2.551430,1588.125645,653.147480,0.899127,404.672506,2.567995,987.270788
min,1.000000,0.000000,2.000000,96.000000,0.000000,0.000000,0.000000,1.000000
25%,5.000000,11.000000,557.000000,494.000000,13.600000,61.000000,0.641912,882.000000
50%,8.000000,13.000000,1212.000000,872.000000,14.000000,158.000000,1.893281,1369.000000
75%,13.000000,15.000000,2311.000000,1236.000000,14.300000,495.000000,4.400004,2040.000000
max,49.000000,23.000000,9756.000000,3764.000000,15.200000,2298.000000,13.963785,7883.000000


In [0]:
selected_features.rename( columns = { 'Pickup - Time' : 'hod'}, inplace = True)

In [0]:
selected_training_features.head()

,Order No,Distance (KM),Pickup - Time,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating,Time from Pickup to Arrival
0,Order_No_4211,4,10,1637,1309,13.8,549,6.058183,745
1,Order_No_27440,20,16,1637,1309,13.8,549,6.058183,2886
2,Order_No_14170,6,13,1637,1309,13.8,549,6.058183,2615
3,Order_No_21727,18,7,1637,1309,13.8,549,6.058183,2986
4,Order_No_8301,7,10,1637,1309,13.8,549,6.058183,1602


In [0]:
uber_sendy_merge = pd.merge(selected_features,uber_sendy, how = 'left', on = ['Order No', 'hod'])

In [0]:
uber_sendy_merge.tail()

,Order No,Distance (KM),hod,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating,Time from Pickup to Arrival,Unnamed: 0,sourceid,dstid,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
28264,Order_No_5207,8,14,2849,2506,14.2,427,5.333408,NaN,601820.0,378.0,266.0,1897.53,1046.86,1671.67,1.66
28265,Order_No_11108,24,15,3504,2234,14.6,1086,10.108850,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28266,Order_No_15696,11,13,417,136,12.8,34,0.141936,NaN,622228.0,169.0,95.0,1418.56,1105.47,1088.76,2.10
28267,Order_No_14332,19,9,34,291,15.0,5,0.641912,NaN,623968.0,378.0,272.0,1589.37,912.96,1435.79,1.53
28268,Order_No_21633,5,13,2931,1170,14.2,586,3.321666,NaN,624501.0,281.0,107.0,755.50,506.56,679.57,1.53


Feature Engineering

In [0]:
combined_features= uber_sendy_merge

combined_features.head()

,Order No,Distance (KM),hod,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating,Time from Pickup to Arrival,Unnamed: 0,sourceid,dstid,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,Order_No_4211,4,10,1637,1309,13.8,549,6.058183,745.0,11.0,386.0,287.0,520.32,529.72,375.88,2.34
1,Order_No_27440,20,16,1637,1309,13.8,549,6.058183,2886.0,26591.0,334.0,88.0,1405.59,733.95,1292.68,1.46
2,Order_No_14170,6,13,1637,1309,13.8,549,6.058183,2615.0,42652.0,110.0,258.0,757.13,615.38,637.81,1.70
3,Order_No_21727,18,7,1637,1309,13.8,549,6.058183,2986.0,42940.0,385.0,102.0,2319.81,909.41,2187.95,1.39
4,Order_No_8301,7,10,1637,1309,13.8,549,6.058183,1602.0,110787.0,151.0,322.0,1485.44,660.96,1406.08,1.35


In [0]:
# Dropping the destination and pickup longitudes and latitudes data to put into the model

model_features = combined_features.drop('Unnamed: 0', axis = 1)
model_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28269 entries, 0 to 28268
Data columns (total 15 columns):
Order No                                    28269 non-null object
Distance (KM)                               28269 non-null int64
hod                                         28269 non-null int64
No_Of_Orders                                28269 non-null int64
Age                                         28269 non-null int64
Average_Rating                              28269 non-null float64
No_of_Ratings                               28269 non-null int64
Rating                                      28269 non-null float64
Time from Pickup to Arrival                 21201 non-null float64
sourceid                                    26885 non-null float64
dstid                                       26885 non-null float64
mean_travel_time                            26885 non-null float64
standard_deviation_travel_time              26885 non-null float64
geometric_mean_travel_time     

In [0]:
model_features_drop  = model_features.dropna()

In [0]:
model_features_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20152 entries, 0 to 21200
Data columns (total 15 columns):
Order No                                    20152 non-null object
Distance (KM)                               20152 non-null int64
hod                                         20152 non-null int64
No_Of_Orders                                20152 non-null int64
Age                                         20152 non-null int64
Average_Rating                              20152 non-null float64
No_of_Ratings                               20152 non-null int64
Rating                                      20152 non-null float64
Time from Pickup to Arrival                 20152 non-null float64
sourceid                                    20152 non-null float64
dstid                                       20152 non-null float64
mean_travel_time                            20152 non-null float64
standard_deviation_travel_time              20152 non-null float64
geometric_mean_travel_time     

In [0]:
model_features.head()

,Order No,Distance (KM),hod,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating,Time from Pickup to Arrival,sourceid,dstid,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,Order_No_4211,4,10,1637,1309,13.8,549,6.058183,745.0,386.0,287.0,520.32,529.72,375.88,2.34
1,Order_No_27440,20,16,1637,1309,13.8,549,6.058183,2886.0,334.0,88.0,1405.59,733.95,1292.68,1.46
2,Order_No_14170,6,13,1637,1309,13.8,549,6.058183,2615.0,110.0,258.0,757.13,615.38,637.81,1.70
3,Order_No_21727,18,7,1637,1309,13.8,549,6.058183,2986.0,385.0,102.0,2319.81,909.41,2187.95,1.39
4,Order_No_8301,7,10,1637,1309,13.8,549,6.058183,1602.0,151.0,322.0,1485.44,660.96,1406.08,1.35


In [0]:
model_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28269 entries, 0 to 28268
Data columns (total 15 columns):
Order No                                    28269 non-null object
Distance (KM)                               28269 non-null int64
hod                                         28269 non-null int64
No_Of_Orders                                28269 non-null int64
Age                                         28269 non-null int64
Average_Rating                              28269 non-null float64
No_of_Ratings                               28269 non-null int64
Rating                                      28269 non-null float64
Time from Pickup to Arrival                 21201 non-null float64
sourceid                                    26885 non-null float64
dstid                                       26885 non-null float64
mean_travel_time                            26885 non-null float64
standard_deviation_travel_time              26885 non-null float64
geometric_mean_travel_time     

Feature Engineering


In [0]:
features_eng_drop = model_features_drop[['Distance (KM)',
'hod','No_Of_Orders',	'Age', 'Average_Rating', 'No_of_Ratings', 
'Rating', 'sourceid',	'dstid'	,'mean_travel_time', 
'standard_deviation_travel_time','geometric_mean_travel_time',	
'geometric_standard_deviation_travel_time', 'Time from Pickup to Arrival']]
features_eng_drop.head()

,Distance (KM),hod,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating,sourceid,dstid,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,Time from Pickup to Arrival
0,4,10,1637,1309,13.8,549,6.058183,386.0,287.0,520.32,529.72,375.88,2.34,745.0
1,20,16,1637,1309,13.8,549,6.058183,334.0,88.0,1405.59,733.95,1292.68,1.46,2886.0
2,6,13,1637,1309,13.8,549,6.058183,110.0,258.0,757.13,615.38,637.81,1.70,2615.0
3,18,7,1637,1309,13.8,549,6.058183,385.0,102.0,2319.81,909.41,2187.95,1.39,2986.0
4,7,10,1637,1309,13.8,549,6.058183,151.0,322.0,1485.44,660.96,1406.08,1.35,1602.0


In [0]:
features_eng_drop = features_eng_drop.dropna()

In [0]:
features_eng_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20152 entries, 0 to 21200
Data columns (total 14 columns):
Distance (KM)                               20152 non-null int64
hod                                         20152 non-null int64
No_Of_Orders                                20152 non-null int64
Age                                         20152 non-null int64
Average_Rating                              20152 non-null float64
No_of_Ratings                               20152 non-null int64
Rating                                      20152 non-null float64
sourceid                                    20152 non-null float64
dstid                                       20152 non-null float64
mean_travel_time                            20152 non-null float64
standard_deviation_travel_time              20152 non-null float64
geometric_mean_travel_time                  20152 non-null float64
geometric_standard_deviation_travel_time    20152 non-null float64
Time from Pickup to Arrival   

In [0]:
pip install category_encoders

In [0]:
import category_encoders as ce

In [0]:
encoder = ce.BinaryEncoder(cols=['sourceid'])

In [0]:
features_eng_drop = encoder.fit_transform(features_eng_drop)

In [0]:
features_eng_drop.head()

,Distance (KM),hod,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating,sourceid_0,sourceid_1,sourceid_2,sourceid_3,sourceid_4,sourceid_5,sourceid_6,sourceid_7,sourceid_8,dstid,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,Time from Pickup to Arrival
0,4,10,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,0,0,1,287.0,520.32,529.72,375.88,2.34,745.0
1,20,16,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,0,1,0,88.0,1405.59,733.95,1292.68,1.46,2886.0
2,6,13,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,0,1,1,258.0,757.13,615.38,637.81,1.70,2615.0
3,18,7,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,1,0,0,102.0,2319.81,909.41,2187.95,1.39,2986.0
4,7,10,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,1,0,1,322.0,1485.44,660.96,1406.08,1.35,1602.0


In [0]:
encoder_1 = ce.BinaryEncoder(cols=['dstid'])

In [0]:
features_eng_drop = encoder_1.fit_transform(features_eng_drop)

In [0]:
features_eng_drop.head()

,Distance (KM),hod,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating,sourceid_0,sourceid_1,sourceid_2,sourceid_3,sourceid_4,sourceid_5,sourceid_6,sourceid_7,sourceid_8,dstid_0,dstid_1,dstid_2,dstid_3,dstid_4,dstid_5,dstid_6,dstid_7,dstid_8,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,Time from Pickup to Arrival
0,4,10,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,520.32,529.72,375.88,2.34,745.0
1,20,16,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1405.59,733.95,1292.68,1.46,2886.0
2,6,13,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,757.13,615.38,637.81,1.70,2615.0
3,18,7,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,2319.81,909.41,2187.95,1.39,2986.0
4,7,10,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,1485.44,660.96,1406.08,1.35,1602.0


In [0]:
encoder_2 = ce.BinaryEncoder(cols=['hod'])

In [0]:
features_eng_drop = encoder_2.fit_transform(features_eng_drop)

In [0]:
features_eng_drop.head()

,Distance (KM),hod_0,hod_1,hod_2,hod_3,hod_4,hod_5,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating,sourceid_0,sourceid_1,sourceid_2,sourceid_3,sourceid_4,sourceid_5,sourceid_6,sourceid_7,sourceid_8,dstid_0,dstid_1,dstid_2,dstid_3,dstid_4,dstid_5,dstid_6,dstid_7,dstid_8,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,Time from Pickup to Arrival
0,4,0,0,0,0,0,1,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,520.32,529.72,375.88,2.34,745.0
1,20,0,0,0,0,1,0,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1405.59,733.95,1292.68,1.46,2886.0
2,6,0,0,0,0,1,1,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,757.13,615.38,637.81,1.70,2615.0
3,18,0,0,0,1,0,0,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,2319.81,909.41,2187.95,1.39,2986.0
4,7,0,0,0,0,0,1,1637,1309,13.8,549,6.058183,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,1485.44,660.96,1406.08,1.35,1602.0


In [0]:
features_eng_drop.describe()

,Distance (KM),hod_0,hod_1,hod_2,hod_3,hod_4,hod_5,No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating,sourceid_0,sourceid_1,sourceid_2,sourceid_3,sourceid_4,sourceid_5,sourceid_6,sourceid_7,sourceid_8,dstid_0,dstid_1,dstid_2,dstid_3,dstid_4,dstid_5,dstid_6,dstid_7,dstid_8,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time,Time from Pickup to Arrival
count,20152.000000,20152.0,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.0,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.0,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000
mean,9.280369,0.0,0.000198,0.330885,0.363537,0.568678,0.630558,1689.192338,980.529674,13.880607,339.568777,2.761748,0.0,0.013051,0.096268,0.341058,0.266872,0.427054,0.354853,0.484617,0.509676,0.0,0.043370,0.154923,0.330935,0.418718,0.532007,0.488785,0.420107,0.616911,1654.672567,830.221917,1486.836758,1.671316,1540.697697
std,5.219902,0.0,0.014088,0.470544,0.481029,0.495273,0.482666,1577.223326,646.562458,0.929560,402.739867,2.555073,0.0,0.113495,0.294966,0.474077,0.442336,0.494663,0.478481,0.499776,0.499919,0.0,0.203695,0.361840,0.470561,0.493361,0.498987,0.499887,0.493588,0.486152,933.316134,361.708783,874.966010,0.317637,959.606977
min,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,96.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,90.070000,138.810000,50.240000,1.060000,1.000000
25%,5.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,557.000000,494.000000,13.600000,60.000000,0.637180,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,895.810000,565.505000,776.410000,1.490000,889.000000
50%,8.000000,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,1209.000000,872.000000,14.000000,158.000000,1.882364,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1510.145000,768.590000,1356.950000,1.600000,1360.500000
75%,12.000000,0.0,0.000000,1.000000,1.000000,1.000000,1.000000,2305.000000,1229.000000,14.300000,491.000000,4.389272,0.0,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2288.020000,1034.150000,2087.700000,1.750000,2002.000000
max,39.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,9756.000000,3764.000000,15.200000,2298.000000,13.963785,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6264.170000,2959.220000,5924.210000,6.610000,7714.000000


**Standardizing** **features**


In [0]:
#data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

# create scaler
scaler = MinMaxScaler()

# fit scaler on numerical data
normalized = scaler.fit_transform(features_eng_drop[['Distance (KM)',
'No_Of_Orders','Age', 'Average_Rating','No_of_Ratings','Rating', 
'mean_travel_time','standard_deviation_travel_time','geometric_mean_travel_time'
,'geometric_standard_deviation_travel_time']])


normalized_model_features = pd.DataFrame(normalized, columns = ['Distance (KM)',
'No_Of_Orders','Age', 'Average_Rating','No_of_Ratings','Rating', 
'mean_travel_time','standard_deviation_travel_time','geometric_mean_travel_time'
,'geometric_standard_deviation_travel_time'])

#'mean_travel_time',	'standard_deviation_travel_time','geometric_mean_travel_time'
normalized_model_features.head()



,Distance (KM),No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,0.078947,0.167624,0.330698,0.907895,0.238903,0.43385,0.069686,0.138600,0.055438,0.230631
1,0.500000,0.167624,0.330698,0.907895,0.238903,0.43385,0.213071,0.211012,0.211516,0.072072
2,0.131579,0.167624,0.330698,0.907895,0.238903,0.43385,0.108042,0.168972,0.100029,0.115315
3,0.447368,0.167624,0.330698,0.907895,0.238903,0.43385,0.361144,0.273223,0.363929,0.059459
4,0.157895,0.167624,0.330698,0.907895,0.238903,0.43385,0.226004,0.185133,0.230822,0.052252


In [0]:
normalized_model_features.to_excel('normalized_lr.xlsx')

In [0]:
files.download('normalized_lr.xlsx')

In [0]:
categorical_features = features_eng_drop[['hod_0','hod_1','hod_2','hod_3',
'hod_4','hod_5','sourceid_0','sourceid_1','sourceid_2','sourceid_3','sourceid_4'
,'sourceid_5','sourceid_6', 'sourceid_7','sourceid_8','dstid_0','dstid_1',
'dstid_2','dstid_3','dstid_4','dstid_5','dstid_6','dstid_7','dstid_8']]

categorical_features.head()

,hod_0,hod_1,hod_2,hod_3,hod_4,hod_5,sourceid_0,sourceid_1,sourceid_2,sourceid_3,sourceid_4,sourceid_5,sourceid_6,sourceid_7,sourceid_8,dstid_0,dstid_1,dstid_2,dstid_3,dstid_4,dstid_5,dstid_6,dstid_7,dstid_8
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1
1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1
3,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1


In [0]:
#Neural network standardized features

normalized_nn = scaler.fit_transform(features_eng_drop[['Distance (KM)',
'No_Of_Orders','Age', 'Average_Rating','No_of_Ratings','Rating'
,'geometric_standard_deviation_travel_time']])


normalized_nn_features = pd.DataFrame(normalized_nn, columns = ['Distance (KM)',
'No_Of_Orders','Age', 'Average_Rating','No_of_Ratings','Rating'
,'geometric_standard_deviation_travel_time'])



neuralnet_feature = pd.concat([normalized_nn_features,categorical_features], axis = 1)
neuralnet_feature.head()

,Distance (KM),No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating,geometric_standard_deviation_travel_time,hod_0,hod_1,hod_2,hod_3,hod_4,hod_5,sourceid_0,sourceid_1,sourceid_2,sourceid_3,sourceid_4,sourceid_5,sourceid_6,sourceid_7,sourceid_8,dstid_0,dstid_1,dstid_2,dstid_3,dstid_4,dstid_5,dstid_6,dstid_7,dstid_8
0,0.078947,0.167624,0.330698,0.907895,0.238903,0.43385,0.230631,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.500000,0.167624,0.330698,0.907895,0.238903,0.43385,0.072072,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.131579,0.167624,0.330698,0.907895,0.238903,0.43385,0.115315,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,0.447368,0.167624,0.330698,0.907895,0.238903,0.43385,0.059459,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.157895,0.167624,0.330698,0.907895,0.238903,0.43385,0.052252,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [0]:
neuralnet_feature.to_excel('neuralnet_feature.xlsx', index = False)

In [0]:
from google.colab import files
files.download("neuralnet_feature.xlsx")

In [0]:
normalized_nn_features.to_excel('normalized_nn_features.xlsx', index = False)

In [0]:
files.download("normalized_nn_features.xlsx")

In [0]:
neuralnet_feature.describe()

,Distance (KM),No_Of_Orders,Age,Average_Rating,No_of_Ratings,Rating,geometric_standard_deviation_travel_time,hod_0,hod_1,hod_2,hod_3,hod_4,hod_5,sourceid_0,sourceid_1,sourceid_2,sourceid_3,sourceid_4,sourceid_5,sourceid_6,sourceid_7,sourceid_8,dstid_0,dstid_1,dstid_2,dstid_3,dstid_4,dstid_5,dstid_6,dstid_7,dstid_8
count,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.0,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.0,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.0,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000,20152.000000
mean,0.217904,0.172974,0.241148,0.913198,0.147767,0.197779,0.110147,0.0,0.000198,0.330885,0.363537,0.568678,0.630558,0.0,0.013051,0.096268,0.341058,0.266872,0.427054,0.354853,0.484617,0.509676,0.0,0.043370,0.154923,0.330935,0.418718,0.532007,0.488785,0.420107,0.616911
std,0.137366,0.161700,0.176271,0.061155,0.175257,0.182979,0.057232,0.0,0.014088,0.470544,0.481029,0.495273,0.482666,0.0,0.113495,0.294966,0.474077,0.442336,0.494663,0.478481,0.499776,0.499919,0.0,0.203695,0.361840,0.470561,0.493361,0.498987,0.499887,0.493588,0.486152
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.105263,0.056900,0.108506,0.894737,0.026110,0.045631,0.077477,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.184211,0.123744,0.211559,0.921053,0.068755,0.134803,0.097297,0.0,0.000000,0.000000,0.000000,1.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
75%,0.289474,0.236108,0.308888,0.940789,0.213664,0.314333,0.124324,0.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Linear regression


In [0]:
from sklearn.linear_model import LinearRegression

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
lm = LinearRegression()

In [0]:
lr_features = normalized_model_features

In [0]:
X_train, X_test, y_train, y_test = train_test_split(lr_features.head(20152),
features_eng_drop['Time from Pickup to Arrival'], test_size=0.3,
 random_state=101)

In [0]:
lm.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
print('Coefficient : \n', lm.coef_)

Coefficient : 
 [ 2594.26493226 -1613.27430697   492.3878368   -520.7342323
  1387.29591663  -636.54789307   124.6349485    -40.39793964
  1306.24381717  -494.78109783]


In [0]:
#lm = LinearRegression()
#lm.fit(X,y)
params = np.append(lm.intercept_,lm.coef_)
predictions = lm.predict(X_train)

#newX = pd.DataFrame({"Constant":np.ones(len(X_train))}).join(pd.DataFrame(X_train))
#MSE = (sum((y_train-predictions)**2))/(len(newX)-len(newX.columns))

# Note if you don't want to use a DataFrame replace the two lines above with
newX = np.append(np.ones((len(X_train),1)), X_train, axis=1)
MSE = (sum((y_train-predictions)**2))/(len(newX)-len(newX[0]))

var_b = MSE*(np.linalg.inv(np.dot(newX.T,newX)).diagonal())
sd_b = np.sqrt(var_b)
ts_b = params/ sd_b

p_values =[2*(1-stats.t.cdf(np.abs(i),(len(newX)-1))) for i in ts_b]

sd_b = np.round(sd_b,3)
ts_b = np.round(ts_b,3)
p_values = np.round(p_values,3)
params = np.round(params,4)

myDF3 = pd.DataFrame()
myDF3["Coefficients"],myDF3["Standard Errors"],myDF3["t values"],myDF3["P-values"] = [params,sd_b,ts_b,p_values]
print(myDF3)


    Coefficients  Standard Errors  t values  P-values
0      1246.2553          107.206    11.625     0.000
1      2594.2649           77.995    33.262     0.000
2     -1613.2743          116.530   -13.844     0.000
3       492.3878           90.266     5.455     0.000
4      -520.7342          110.023    -4.733     0.000
5      1387.2959          132.661    10.457     0.000
6      -636.5479          112.000    -5.683     0.000
7       124.6349         1341.534     0.093     0.926
8       -40.3979          184.265    -0.219     0.826
9      1306.2438         1283.141     1.018     0.309
10     -494.7811          197.918    -2.500     0.012


In [0]:
pip install scipy

In [0]:
import statsmodels.api as sm

In [0]:
from scipy import stats

In [0]:
X2 = sm.add_constant(X_train)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [0]:
est = sm.OLS(y_train, X2)

In [0]:
est2 = est.fit()

In [0]:
print(est2.summary())

                                 OLS Regression Results                                
Dep. Variable:     Time from Pickup to Arrival   R-squared:                       0.316
Model:                                     OLS   Adj. R-squared:                  0.316
Method:                          Least Squares   F-statistic:                     1330.
Date:                         Mon, 14 Oct 2019   Prob (F-statistic):               0.00
Time:                                 11:31:26   Log-Likelihood:            -1.6314e+05
No. Observations:                        20152   AIC:                         3.263e+05
Df Residuals:                            20144   BIC:                         3.264e+05
Df Model:                                    7                                         
Covariance Type:                     nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
------------------------------

In [0]:
prediction_test = lm.predict(X_test)

In [0]:
predictions_series_test = pd.Series(data = prediction_test, index = None)

In [0]:
prediction_train = lm.predict(X_train)

In [0]:
predictions_series_train = pd.Series(data = prediction_train, index = None)

In [0]:
from sklearn import metrics

In [0]:
print('MAE:', metrics.mean_absolute_error
      (y_train, prediction_train))
print('MSE:', metrics.mean_squared_error
      (y_train, prediction_train))
print('RMSE: ', np.sqrt(metrics.mean_squared_error
      (y_train, prediction_train)))

MAE: 538.7039130454542
MSE: 619843.7991298644
RMSE:  787.3015935014132


In [0]:
print('MAE:', metrics.mean_absolute_error
      (y_test, prediction_test))
print('MSE:', metrics.mean_squared_error
      (y_test, prediction_test))
print('RMSE: ', np.sqrt(metrics.mean_squared_error
      (y_test, prediction_test)))

MAE: 540.6590178767856
MSE: 597422.967106382
RMSE:  772.9314116442558


Building a Neural Network Model using Keras

In [0]:
pip install -q pyyaml

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

from tensorflow.keras import layers

print(tf.version.VERSION)
print(tf.keras.__version__)

1.15.0
2.2.4-tf


In [0]:
lrelu = lambda x: tf.keras.activations.relu(x, alpha =0.1)



model = tf.keras.Sequential()
# Adds a densely-connected layer with 6 units to the model:
model.add(layers.Dense(80, activation = lrelu, 
kernel_initializer = 
tf.keras.initializers.VarianceScaling(scale = 1,
mode = 'fan_in', distribution = 'normal'),
bias_initializer = tf.keras.initializers.VarianceScaling(scale = 1,
mode = 'fan_in', distribution = 'normal'),
kernel_regularizer = tf.keras.regularizers.l2(3),
bias_regularizer = tf.keras.regularizers.l2(1.775)))
# Add another layer with 6 units:

model.add(layers.Dense(20, activation= lrelu,
kernel_initializer= tf.keras.initializers.VarianceScaling(scale = 1
,mode = 'fan_in', distribution = 'normal'),
bias_initializer = tf.keras.initializers.VarianceScaling(scale = 1,
mode = 'fan_in', distribution = 'normal'),
kernel_regularizer = tf.keras.regularizers.l2(3),
bias_regularizer = tf.keras.regularizers.l2(1.775)))

# Add a linear layer with 4 output unit:
model.add(layers.Dense(1, activation= 'linear',
kernel_initializer= tf.keras.initializers.VarianceScaling(scale = 1,
mode = 'fan_in', distribution = 'normal'),
bias_initializer = tf.keras.initializers.VarianceScaling(scale = 1,
mode = 'fan_in', distribution = 'normal'),
kernel_regularizer = tf.keras.regularizers.l2(3),
bias_regularizer = tf.keras.regularizers.l2(1.775)))



Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


In [0]:
model.compile(optimizer=tf.train.AdamOptimizer(0.0015),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error
1


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


1

In [0]:
training_features = np.array(neuralnet_feature.head(20152))
target_train_variables = np.array(model_features_drop['Time from Pickup to Arrival'])

In [0]:
model.fit(training_features, target_train_variables , validation_split = 0.1, epochs = 500, batch_size = 15)

Train on 18136 samples, validate on 2016 samples
Epoch 1/500
18136/18136 [==============================] - 2s 127us/sample - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/500
18136/18136 [==============================] - 2s 100us/sample - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 3/500
18136/18136 [==============================] - 2s 99us/sample - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 4/500
18136/18136 [==============================] - 2s 99us/sample - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 5/500
18136/18136 [==============================] - 2s 97us/sample - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 6/500
18136/18136 [==============================] - 2s 97us/sample - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_

KeyboardInterrupt: ignored

In [0]:
evaluating_features = np.array(standardized_features_eng_drop)

In [0]:
# Evaluation of the model
model.evaluate(training_features, target_train_variables, batch_size = 1)

20152/20152 [==============================] - 9s 458us/sample - loss: 563211.5302 - mean_absolute_error: 524.5867


[563211.5302207617, 524.58673]

In [0]:
#Prediction of the model
result = model.predict(evaluating_features, batch_size = 1)

In [0]:
loss_and_metrics = model.evaluate(training_features, target_train_variables, batch_size=10)

classes = model.predict(x_test, batch_size=1)

test = training_features.reshape(-1)

plt.plot(test ,classes , c = 'r')
plt.plot(test, target_train_variables , c = 'b')
plt.show()

In [0]:
result = pd.DataFrame(result, columns = ['Time from Pickup to Arrival'])
result.head()

,Time from Pickup to Arrival
0,678.655640
1,2132.711670
2,1345.699829
3,1858.996216
4,1413.233643


In [0]:
dataset = pd.concat([features_train_new['Order No'],features_test_new['Order No']],ignore_index = True)
dataset_series = pd.DataFrame(data=dataset,index = None, columns = ['Order No'])
dataset_series.head(5)

,Order No
0,Order_No_4211
1,Order_No_27440
2,Order_No_14170
3,Order_No_21727
4,Order_No_8301


In [0]:
final_result = pd.concat([dataset_series, result], axis = 1)
final_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28269 entries, 0 to 28268
Data columns (total 2 columns):
Order No                       28269 non-null object
Time from Pickup to Arrival    26885 non-null float32
dtypes: float32(1), object(1)
memory usage: 331.4+ KB


In [0]:
final_result.to_csv('final_result.csv', sep=',', index = False)

In [0]:
from google.colab import files
files.download("final_result.csv")